In [1]:

%load_ext autoreload

%autoreload 1

In [2]:
import xarray as xr
from pathlib import Path
import numpy as np
import numexpr as ne
import dask

import time
from analysis_utils import load_data_set, calc_temp_spac_means

%aimport analysis_utils

In [3]:
job = "20220314T200911"
job_path = Path("/users/PAS0409/madelinescyphers/palm/current_version/JOBS") / job

# job_path = Path("/Users/madelinescyphers/projs/palm/data/20220308T183847")
job_num = "002."

data_path = job_path / f"OUTPUT/{job}_3d.{job_num}nc"
ts_path = job_path / f"OUTPUT/{job}_ts.{job_num}nc"
xy_path = job_path / f"OUTPUT/{job}_xy.{job_num}nc"
pr_path = job_path / f"OUTPUT/{job}_pr.{job_num}nc"
av_3d_path = job_path / f"OUTPUT/{job}_av_3d.{job_num}nc"
lad_path = job_path / f"INPUT/{job}_static"
ds = load_data_set(data_path, chunk=False)
ds_ts = xr.open_dataset(ts_path)
# # ds_pr = xr.open_dataset(pr_path)
# ds_3d_av = xr.open_dataset(av_3d_path)
ds_xy = xr.open_dataset(xy_path)
ds_lad = xr.open_dataset(lad_path)
ds = ds.rename({"zu_3d": "z"})


In [38]:
ds_lad = ds_lad.rename({"zlad": "z"})
ds = ds.drop_isel(z=[0, 1, -1], x=-1, y=-1)

ds_3d_av = ds_3d_av.rename({"zu_3d": "z"})
ds_3d_av = ds_3d_av.interp(xu=ds_3d_av.x, yv=ds_3d_av.y, zw_3d=ds_3d_av.z)

NameError: name 'ds_3d_av' is not defined

In [59]:
ds_pr.zwu

<xarray.DataArray 'zwu' (zwu: 194)>
array([  1.5,   3. ,   6. ,   9. ,  12. ,  15. ,  18. ,  21. ,  24. ,  27. ,
        30. ,  33. ,  36. ,  39. ,  42. ,  45. ,  48. ,  51. ,  54. ,  57. ,
        60. ,  63. ,  66. ,  69. ,  72. ,  75. ,  78. ,  81. ,  84. ,  87. ,
        90. ,  93. ,  96. ,  99. , 102. , 105. , 108. , 111. , 114. , 117. ,
       120. , 123. , 126. , 129. , 132. , 135. , 138. , 141. , 144. , 147. ,
       150. , 153. , 156. , 159. , 162. , 165. , 168. , 171. , 174. , 177. ,
       180. , 183. , 186. , 189. , 192. , 195. , 198. , 201. , 204. , 207. ,
       210. , 213. , 216. , 219. , 222. , 225. , 228. , 231. , 234. , 237. ,
       240. , 243. , 246. , 249. , 252. , 255. , 258. , 261. , 264. , 267. ,
       270. , 273. , 276. , 279. , 282. , 285. , 288. , 291. , 294. , 297. ,
       300. , 303. , 306. , 309. , 312. , 315. , 318. , 321. , 324. , 327. ,
       330. , 333. , 336. , 339. , 342. , 345. , 348. , 351. , 354. , 357. ,
       360. , 363. , 366. , 369. , 372. , 375. , 378. , 381. , 384. , 387. ,
       390. , 393. , 396. , 399. , 402. , 405. , 408. , 411. , 414. , 417. ,
       420. , 423. , 426. , 429. , 432. , 435. , 438. , 441. , 444. , 447. ,
       450. , 453. , 456. , 459. , 462. , 465. , 468. , 471. , 474. , 477. ,
       480. , 483. , 486. , 489. , 492. , 495. , 498. , 501. , 504. , 507. ,
       510. , 513. , 516. , 519. , 522. , 525. , 528. , 531. , 534. , 537. ,
       540. , 543. , 546. , 549. , 552. , 555. , 558. , 561. , 564. , 567. ,
       570. , 573. , 576. , 579. ])
Coordinates:
  * zwu      (zwu) float64 1.5 3.0 6.0 9.0 12.0 ... 570.0 573.0 576.0 579.0
Attributes:
    units:    meters

In [72]:
job_num = ""
pr_path = job_path / f"OUTPUT/{job}_pr.{job_num}nc"
ds_pr = xr.open_dataset(pr_path)


ds_pr = ds_pr.rename({"zwv": "z"})
ds_pr = ds_pr.interp(zwu=ds_pr.z)

In [40]:
dz = 3

z_scalar  = np.arange(150//dz, 300//dz)

urban_ratio = 0.5
y_domain = np.arange(int(ds.s.y.size * urban_ratio), ds.s.y.size)

In [41]:
ds = ds.interp(xu=ds.x, yv=ds.y, zw_3d=ds.z)

In [42]:
ds.u * ds.u

<xarray.DataArray 'u' (time: 2, z: 191, y: 431, x: 191)>
array([[[[6.78163229e-06, 2.46535190e-03, 1.19398496e-02, ...,
          2.09509866e-02, 6.75951020e-03, 1.98037391e-03],
         [3.11993641e-08, 1.80125725e-04, 2.00398988e-03, ...,
          5.44244377e-03, 3.63655715e-04, 6.38618501e-05],
         [4.33345718e-05, 9.98680393e-04, 2.51524945e-03, ...,
          2.96411710e-03, 3.04473868e-03, 2.84600346e-03],
         ...,
         [5.03105982e-06, 6.59180311e-04, 1.32116236e-03, ...,
          6.92056624e-04, 7.02819863e-04, 1.82468214e-04],
         [1.58407228e-05, 2.43902890e-04, 6.56642523e-04, ...,
          2.54307987e-03, 1.30051368e-03, 1.21972979e-03],
         [8.47137575e-05, 2.39109658e-03, 7.43457833e-03, ...,
          1.31900689e-02, 8.10495678e-03, 5.41905738e-03]],

        [[2.77939556e-03, 2.25372785e-03, 4.01840306e-03, ...,
          1.88680650e-03, 1.75953991e-04, 3.36306664e-04],
         [5.62414152e-04, 5.90416799e-04, 1.48930261e-03, ...,
          8.90864787e-04, 3.14512189e-03, 6.54160508e-04],
         [1.11607500e-03, 7.98537699e-04, 6.53234856e-04, ...,
          1.46047821e-02, 1.15689022e-02, 2.16961119e-03],
...
         [1.14920897e-11, 1.10197499e-11, 1.05342998e-11, ...,
          1.32082614e-11, 1.28092435e-11, 1.23888958e-11],
         [1.15419487e-11, 1.10744932e-11, 1.05935650e-11, ...,
          1.32354060e-11, 1.28424978e-11, 1.24279888e-11],
         [1.15964772e-11, 1.11337834e-11, 1.06572492e-11, ...,
          1.32676424e-11, 1.28807481e-11, 1.24719744e-11]],

        [[1.14051532e-11, 1.09499634e-11, 1.04811809e-11, ...,
          1.30494104e-11, 1.26687145e-11, 1.22665117e-11],
         [1.14721016e-11, 1.10211410e-11, 1.05562077e-11, ...,
          1.30960258e-11, 1.27208850e-11, 1.23239379e-11],
         [1.15437378e-11, 1.10969002e-11, 1.06357053e-11, ...,
          1.31476812e-11, 1.27780187e-11, 1.23862414e-11],
         ...,
         [1.11838772e-11, 1.07106489e-11, 1.02252899e-11, ...,
          1.29133213e-11, 1.25095511e-11, 1.20853974e-11],
         [1.12322834e-11, 1.07637382e-11, 1.02827051e-11, ...,
          1.29398259e-11, 1.25419602e-11, 1.21234413e-11],
         [1.12852705e-11, 1.08212912e-11, 1.03444614e-11, ...,
          1.29713214e-11, 1.25792679e-11, 1.21662832e-11]]]])
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.251000 00:00:10.497000
  * z        (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.286e+03 1.288e+03 1.292e+03
    xu       (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
    yv       (y) float64 1.5 4.5 7.5 10.5 ... 1.286e+03 1.288e+03 1.292e+03
    zw_3d    (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5

In [43]:
ds.uu

<xarray.DataArray 'uu' (time: 2, z: 191, y: 431, x: 191)>
array([[[[3.23444649e-04, 3.65294254e-03, 1.25726680e-02, ...,
          2.18481282e-02, 7.82070635e-03, 2.00678164e-03],
         [9.67890998e-07, 3.30841376e-04, 2.36758712e-03, ...,
          6.36375602e-03, 9.56579635e-04, 7.12118672e-05],
         [2.12714527e-04, 1.14278594e-03, 2.55809911e-03, ...,
          2.96946138e-03, 3.04722355e-03, 2.84606835e-03],
         ...,
         [6.03180997e-05, 9.15051700e-04, 1.34949340e-03, ...,
          7.09008542e-04, 7.18135241e-04, 2.65081384e-04],
         [3.22814944e-04, 2.48215787e-04, 7.19541655e-04, ...,
          2.75778910e-03, 1.30059588e-03, 1.22175942e-03],
         [1.15758169e-03, 3.03362717e-03, 7.57802627e-03, ...,
          1.34914224e-02, 8.16062302e-03, 5.49920159e-03]],

        [[2.77999439e-03, 2.27373309e-03, 4.43416170e-03, ...,
          2.70918640e-03, 1.78190385e-04, 3.79463119e-04],
         [5.78918582e-04, 6.02522618e-04, 1.80515848e-03, ...,
          1.64084415e-03, 3.14644841e-03, 1.51576521e-03],
         [1.13313037e-03, 7.99577101e-04, 6.56058866e-04, ...,
          1.46095934e-02, 1.18086813e-02, 4.23939992e-03],
...
         [1.14932660e-11, 1.10210530e-11, 1.05357324e-11, ...,
          1.32089725e-11, 1.28100638e-11, 1.23898300e-11],
         [1.15430942e-11, 1.10757636e-11, 1.05949637e-11, ...,
          1.32360941e-11, 1.28432924e-11, 1.24288952e-11],
         [1.15975935e-11, 1.11350235e-11, 1.06586168e-11, ...,
          1.32683053e-11, 1.28815170e-11, 1.24728552e-11]],

        [[1.14062509e-11, 1.09511835e-11, 1.04825263e-11, ...,
          1.30500641e-11, 1.26694727e-11, 1.22673790e-11],
         [1.14731727e-11, 1.10223324e-11, 1.05575227e-11, ...,
          1.30966574e-11, 1.27216193e-11, 1.23247792e-11],
         [1.15447816e-11, 1.10980634e-11, 1.06369904e-11, ...,
          1.31482906e-11, 1.27787290e-11, 1.23870580e-11],
         ...,
         [1.11850928e-11, 1.07119912e-11, 1.02267621e-11, ...,
          1.29140687e-11, 1.25104089e-11, 1.20863705e-11],
         [1.12334699e-11, 1.07650494e-11, 1.02841442e-11, ...,
          1.29405488e-11, 1.25427919e-11, 1.21243870e-11],
         [1.12864267e-11, 1.08225716e-11, 1.03458687e-11, ...,
          1.29720210e-11, 1.25800741e-11, 1.21672017e-11]]]])
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.251000 00:00:10.497000
  * z        (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.286e+03 1.288e+03 1.292e+03
    xu       (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
    yv       (y) float64 1.5 4.5 7.5 10.5 ... 1.286e+03 1.288e+03 1.292e+03
    zw_3d    (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5
Attributes:
    units:      m2/s2
    long_name:  uu

In [44]:
tot = ds.uu + ds.vv + ds.ww

In [45]:
ubar = tot**(1/2)

In [46]:
ubar

<xarray.DataArray (time: 2, z: 191, y: 431, x: 191)>
array([[[[3.92437042, 3.90730303, 3.97072394, ..., 3.94700477,
          3.91416195, 3.87463037],
         [3.95097963, 3.94642167, 4.01445717, ..., 3.99980739,
          3.95903019, 3.8843339 ],
         [3.95470051, 3.95966443, 4.02743755, ..., 4.03290508,
          3.97275926, 3.88196099],
         ...,
         [3.81759304, 3.81654521, 3.83600486, ..., 3.73672189,
          3.82727604, 3.8679447 ],
         [3.82282386, 3.81147168, 3.85854089, ..., 3.78319782,
          3.84118138, 3.86675824],
         [3.84573006, 3.82421715, 3.8857469 , ..., 3.84459127,
          3.85838867, 3.86651218]],

        [[4.04279142, 4.03144797, 4.04893047, ..., 4.06352818,
          4.03466191, 4.03814199],
         [4.05462479, 4.05476696, 4.12297893, ..., 4.11743766,
          4.06189156, 4.04568777],
         [4.04804909, 4.05041469, 4.15478645, ..., 4.15285283,
          4.06644052, 4.02545118],
...
         [3.99946812, 3.99946812, 3.99946812, ..., 3.99946812,
          3.99946812, 3.99946812],
         [3.99946949, 3.99946949, 3.99946949, ..., 3.99946949,
          3.99946949, 3.99946949],
         [3.99947104, 3.99947104, 3.99947104, ..., 3.99947104,
          3.99947104, 3.99947104]],

        [[3.99948457, 3.99948457, 3.99948457, ..., 3.99948463,
          3.99948457, 3.99948457],
         [3.99948648, 3.99948648, 3.99948648, ..., 3.99948654,
          3.99948648, 3.99948648],
         [3.99948856, 3.99948856, 3.9994885 , ..., 3.99948856,
          3.99948856, 3.99948856],
         ...,
         [3.99947843, 3.99947843, 3.99947837, ..., 3.99947843,
          3.99947843, 3.99947843],
         [3.99947974, 3.99947974, 3.99947974, ..., 3.99947974,
          3.99947974, 3.99947974],
         [3.99948123, 3.99948123, 3.99948123, ..., 3.99948123,
          3.99948123, 3.99948123]]]])
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.251000 00:00:10.497000
  * z        (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.286e+03 1.288e+03 1.292e+03
    xu       (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
    yv       (y) float64 1.5 4.5 7.5 10.5 ... 1.286e+03 1.288e+03 1.292e+03
    zw_3d    (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5

In [47]:
ubar.isel(y=np.arange(0, 50)).mean(skipna=True)

<xarray.DataArray ()>
array(3.96268338)

In [48]:
ds_ts["us*_00"]

<xarray.DataArray 'us*_00' (time: 18)>
array([0.066632, 0.101849, 0.13183 , 0.156781, 0.179038, 0.199685, 0.217894,
       0.236597, 0.251891, 0.265423, 0.276901, 0.285021, 0.293606, 0.298048,
       0.300634, 0.299417, 0.294873, 0.294414], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:00.635000 ... 00:00:10.497000
Attributes:
    units:      m/s
    long_name:  us* SR 0

In [49]:
ds_ts["us*_01"]

<xarray.DataArray 'us*_01' (time: 18)>
array([0.00761 , 0.011446, 0.014702, 0.017354, 0.019762, 0.021874, 0.023988,
       0.025984, 0.027639, 0.029199, 0.030541, 0.031736, 0.032838, 0.03367 ,
       0.034232, 0.034657, 0.034883, 0.03529 ], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:00.635000 ... 00:00:10.497000
Attributes:
    units:      m/s
    long_name:  us* SR 0

In [50]:
((ds_ts.E_01 * 2) ** (1/2))

<xarray.DataArray 'E_01' (time: 18)>
array([1.3655287, 1.3642672, 1.3632911, 1.3624634, 1.3617187, 1.3610379,
       1.3604069, 1.3598205, 1.3592662, 1.3587431, 1.3582464, 1.3577788,
       1.3573407, 1.3569313, 1.3565542, 1.3562117, 1.3559067, 1.3556399],
      dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:00.635000 ... 00:00:10.497000

In [56]:
ds_pr

<xarray.Dataset>
Dimensions:       (time: 1, zw"u": 194, zw*u*: 194, zwu: 194, zw"v": 194,
                   zw*v*: 194, zwv: 194)
Coordinates:
  * time          (time) timedelta64[ns] 00:00:50.547000
  * zw"u"         (zw"u") float64 1.5 3.0 6.0 9.0 ... 570.0 573.0 576.0 579.0
  * zw*u*         (zw*u*) float64 0.0 3.0 6.0 9.0 ... 570.0 573.0 576.0 579.0
  * zwu           (zwu) float64 1.5 3.0 6.0 9.0 12.0 ... 570.0 573.0 576.0 579.0
  * zw"v"         (zw"v") float64 1.5 3.0 6.0 9.0 ... 570.0 573.0 576.0 579.0
  * zw*v*         (zw*v*) float64 0.0 3.0 6.0 9.0 ... 570.0 573.0 576.0 579.0
  * zwv           (zwv) float64 1.5 3.0 6.0 9.0 12.0 ... 570.0 573.0 576.0 579.0
Data variables: (12/13)
    w"u"          (time, zw"u") float32 ...
    w*u*          (time, zw*u*) float32 ...
    wu            (time, zwu) float32 2.603e-05 6.72e-05 ... 2.144e-12 0.0
    w"v"          (time, zw"v") float32 ...
    w*v*          (time, zw*v*) float32 ...
    wv            (time, zwv) float32 -0.05472 -0.08628 ... -9.022e-08 0.0
    ...            ...
    NORM_ws2      (time) float32 ...
    NORM_tsw2     (time) float32 ...
    NORM_ws3      (time) float32 ...
    NORM_ws2tsw   (time) float32 ...
    NORM_wstsw2   (time) float32 ...
    NORM_z_i      (time) float32 ...
Attributes: (12/31)
    title:           PALM 6.0  Rev: 4359M  run: 20220314T200911.00  host: def...
    Conventions:     CF-1.7
    creation_time:   2022-03-14 20:11:40 -04
    data_content:    podsprav
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    comment:         
    time_avg:           50.0 s avg
    no_rows:         3
    no_columns:      2
    cross_profiles:  ;w"u",w*u*,wu,w"v",w*v*,wv;
    VAR_LIST:        ;w"u";w*u*;wu;w"v";w*v*;wv;

In [52]:
ubar_z_prof = ubar.mean(dim=["x", "y", "time"], skipna=True)

In [75]:
ustar = (ds_pr.wu**2 + ds_pr.wv**2)**(1/4)

In [370]:
ds_3d_av

<xarray.Dataset>
Dimensions:  (time: 1, z: 194, x: 192, y: 432, zpc_3d: 6)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:50.064000
  * z        (z) float64 0.0 1.5 4.5 7.5 10.5 ... 565.5 568.5 571.5 574.5 577.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
  * zpc_3d   (zpc_3d) float64 0.0 1.5 4.5 7.5 10.5 13.5
    xu       (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
    yv       (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zw_3d    (z) float64 0.0 1.5 4.5 7.5 10.5 ... 565.5 568.5 571.5 574.5 577.5
Data variables: (12/18)
    E_UTM    (x) float64 3.154e+05 3.154e+05 3.154e+05 ... 3.16e+05 3.16e+05
    N_UTM    (y) float64 4.448e+06 4.448e+06 4.448e+06 ... 4.449e+06 4.449e+06
    Eu_UTM   (x) float64 3.154e+05 3.154e+05 3.154e+05 ... 3.16e+05 3.16e+05 nan
    Nu_UTM   (y) float64 4.448e+06 4.448e+06 4.448e+06 ... 4.449e+06 4.449e+06
    Ev_UTM   (x) float64 3.154e+05 3.154e+05 3.154e+05 ... 3.16e+05 3.16e+05
    Nv_UTM   (y) float64 4.448e+06 4.448e+06 4.448e+06 ... 4.449e+06 nan
    ...       ...
    crs      int32 0
    w        (time, z, y, x) float64 nan nan nan ... -0.002078 -0.002078
    u        (time, z, y, x) float64 nan nan nan ... -2.598e-06 -2.411e-06 nan
    v        (time, z, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    wv       (time, z, y, x) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    wu       (time, z, y, x) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
Attributes: (12/28)
    title:           PALM 6.0  Rev: 4359M  run: 20220308T183847.00  host: def...
    Conventions:     CF-1.7
    creation_time:   2022-03-14 10:13:18 -04
    data_content:    3d_av
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    references:      
    keywords:        
    licence:         
    comment:         
    time_avg:           50.0 s avg
    VAR_LIST:        ;w;u;v;wv;wu;

In [362]:
ds_3d_av.u.mean(dim=["x", "y"])*ds_3d_av.w.mean(dim=["x", "y"])

<xarray.DataArray (time: 1, z: 194)>
array([[            nan,             nan, -3.30415109e-16,
         1.26304570e-16,  2.50918706e-17,  2.84883577e-15,
        -2.03509212e-15,  3.11860368e-16, -4.19804834e-15,
        -3.71494494e-15, -2.03146737e-15,  2.38479553e-17,
        -3.42183742e-15, -2.42903744e-15, -1.36773855e-15,
         1.09996656e-15,  6.86345548e-17,  4.86129671e-17,
        -7.58459206e-17, -1.33067669e-17, -1.79861149e-16,
         2.42520558e-15, -1.50324930e-17, -4.27044401e-16,
        -5.04327162e-16,  1.65312672e-15,  2.90818200e-15,
         7.20878029e-16,  6.60028742e-17,  7.81770213e-16,
        -1.22104726e-15, -6.46807772e-16, -2.38087342e-16,
        -3.49528352e-16, -2.57313813e-15,  1.81103108e-15,
         1.64457253e-15,  1.73244515e-17, -3.89475503e-16,
        -1.63763236e-15,  2.47777456e-16,  4.13173920e-16,
         6.22884022e-16,  1.16206445e-19,  4.69225051e-16,
        -2.74601237e-16,  2.03519331e-16, -3.34622814e-16,
         1.18244992e-15,  1.02995761e-15, -2.85876042e-15,
        -4.40714883e-16,  6.74158605e-15, -1.62359085e-16,
        -3.94346076e-16, -2.80682220e-17, -6.18463374e-16,
         1.68483298e-16, -1.19728168e-16,  2.96176973e-16,
...
         2.47996606e-20, -1.18640850e-21, -1.01854362e-21,
         6.92121426e-21,  1.77509693e-20,  5.30125380e-21,
        -7.96948903e-21, -3.43963307e-21, -5.00664697e-21,
        -1.33164833e-20, -2.73294814e-20, -1.28529991e-20,
         1.20364372e-20,  1.21999737e-20, -6.65255013e-21,
        -1.41179952e-20, -1.55686349e-20, -1.29714092e-20,
         4.55543464e-21, -4.09721257e-21,  7.35766216e-22,
         1.70100623e-20,  1.02627075e-20, -3.19212004e-21,
        -2.63057002e-21,  8.22005835e-21, -2.93527104e-21,
         2.30875070e-21,  9.69922213e-21, -1.80094248e-21,
         3.39458794e-21,  9.73007690e-21,  4.01955363e-22,
        -7.25102570e-21, -2.89478190e-21,  2.72099869e-22,
         1.07268714e-21,  2.68550167e-21,  2.45431054e-21,
         3.82825079e-21,  4.78135247e-21,  3.06425193e-21,
         1.95795419e-21, -1.30585660e-21, -2.60703250e-21,
         6.97424858e-22, -2.53313574e-21, -3.39058812e-21,
        -1.70780184e-21, -3.91295539e-21, -4.34419421e-21,
        -3.54491614e-21, -1.03856204e-21,  2.65221629e-21,
         2.21854887e-21,  1.50797507e-21, -1.95358593e-22,
        -3.96652397e-22,  8.50846843e-22]])
Coordinates:
  * time     (time) timedelta64[ns] 00:00:50.064000
  * z        (z) float64 0.0 1.5 4.5 7.5 10.5 ... 565.5 568.5 571.5 574.5 577.5
    zw_3d    (z) float64 0.0 1.5 4.5 7.5 10.5 ... 565.5 568.5 571.5 574.5 577.5

In [375]:
# ds_3d_av.wu.mean(dim=["x", "y"]) - ds_3d_av.u.mean(dim=["x", "y"])*ds_3d_av.w.mean(dim=["x", "y"])
(ds_3d_av.wu - ds_3d_av.u * ds_3d_av.w).mean(skipna=True)

<xarray.DataArray ()>
array(2.77990659e-05)

In [350]:
ds.w.mean(dim=["x", "y", "time"]) * ds.u.mean(dim=["x", "y", "time"])

<xarray.DataArray (z: 191)>
array([ 1.27892413e-08, -5.49498355e-09, -7.55342730e-11,  2.89748387e-08,
        8.28384063e-08, -6.59764230e-08, -1.66093002e-07, -1.13379842e-07,
        1.40415485e-07, -5.22973535e-08, -1.82746796e-07, -2.46362747e-07,
       -2.60403895e-07, -1.64124943e-07,  1.12599879e-08,  1.62835659e-07,
        6.91178519e-09,  4.80792062e-09,  1.27726112e-07,  1.54521637e-07,
        2.00327435e-08,  5.31724506e-08,  2.83700077e-07,  1.58457196e-07,
        2.45145523e-07,  7.91231364e-08,  9.69603818e-08,  7.57090939e-08,
       -4.47174136e-08, -1.11164747e-07, -9.78543869e-08,  7.25911145e-08,
        2.27296964e-07,  2.73408151e-07,  1.48674645e-07, -8.88444121e-08,
       -1.48283750e-08, -5.97325386e-08, -1.29472663e-07, -6.43048739e-08,
       -4.90036088e-08, -3.19348381e-08,  6.67302554e-08,  3.16234958e-08,
       -1.85391114e-08, -1.92106107e-08,  6.66665155e-08,  2.37077563e-07,
        1.48075150e-07,  9.39050123e-08,  1.87821114e-07,  1.26156608e-07,
        5.40778662e-08,  4.05367681e-08,  5.69459848e-08,  1.01812509e-08,
        2.97332354e-08, -3.57864550e-08,  3.95713435e-09,  4.82977584e-09,
       -4.49233232e-08, -5.46724330e-08, -1.23138086e-08,  8.41192868e-09,
        1.78908863e-09, -3.22841578e-10, -1.82215369e-11,  3.52781460e-11,
        1.91475350e-11,  1.78183796e-11,  1.99603727e-11,  2.10205374e-11,
        2.15976063e-11,  2.18765188e-11,  2.18859493e-11,  2.16779149e-11,
        2.13016097e-11,  2.07976185e-11,  2.01994330e-11,  1.95343830e-11,
...
        3.71491252e-12,  3.52840421e-12,  3.35166762e-12,  3.18417531e-12,
        3.02542882e-12,  2.87495760e-12,  2.73231713e-12,  2.59708777e-12,
        2.46887329e-12,  2.34729987e-12,  2.23201436e-12,  2.12268344e-12,
        2.01899259e-12,  1.92064461e-12,  1.82735945e-12,  1.73887264e-12,
        1.65491051e-12,  1.57523905e-12,  1.50010627e-12,  1.42688513e-12,
        1.35375808e-12,  1.28336934e-12,  1.21691669e-12,  1.15397137e-12,
        1.09435572e-12,  1.03789246e-12,  9.84412934e-13,  9.33757671e-13,
        8.85776245e-13,  8.40326493e-13,  7.97274097e-13,  7.56492149e-13,
        7.17860599e-13,  6.81266103e-13,  6.46601413e-13,  6.13765062e-13,
        5.82661099e-13,  5.53198759e-13,  5.25292137e-13,  4.98859912e-13,
        4.73825128e-13,  4.50114889e-13,  4.27660197e-13,  4.06395683e-13,
        3.86259428e-13,  3.67192758e-13,  3.49140060e-13,  3.32048651e-13,
        3.15868553e-13,  3.00552396e-13,  2.86055254e-13,  2.72334499e-13,
        2.59349693e-13,  2.47062455e-13,  2.35436349e-13,  2.24436809e-13,
        2.14030964e-13,  2.04187604e-13,  1.94877085e-13,  1.86071205e-13,
        1.77743158e-13,  1.69867446e-13,  1.62419798e-13,  1.55377107e-13,
        1.48717372e-13,  1.42419612e-13,  1.36463840e-13,  1.30830964e-13,
        1.25502783e-13,  1.20461882e-13,  1.15691623e-13,  1.11176092e-13,
        1.06900032e-13,  1.02848829e-13,  9.90084448e-14,  9.53653771e-14,
        9.19069660e-14,  8.86190250e-14,  8.54692951e-14])
Coordinates:
  * z        (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5
    zw_3d    (z) float64 4.5 7.5 10.5 13.5 16.5 ... 565.5 568.5 571.5 574.5

In [369]:
ds_pr.drop_isel(z=[0, 1, -1]).wu.mean()

<xarray.DataArray 'wu' ()>
array(3.8484936e-06, dtype=float32)

In [62]:
ustar

<xarray.DataArray (time: 1, zwu: 194, z: 194)>
array([[[2.3392391e-01, 2.9374105e-01, 9.3837760e-02, ...,
         5.1022498e-03, 5.1022498e-03, 5.1022344e-03],
        [2.3392399e-01, 2.9374111e-01, 9.3838923e-02, ...,
         8.1977695e-03, 8.1977695e-03, 8.1977658e-03],
        [2.3392434e-01, 2.9374126e-01, 9.3844265e-02, ...,
         1.2204269e-02, 1.2204269e-02, 1.2204267e-02],
        ...,
        [2.3392391e-01, 2.9374105e-01, 9.3837559e-02, ...,
         3.0044874e-04, 3.0036236e-04, 1.4648350e-06],
        [2.3392391e-01, 2.9374105e-01, 9.3837559e-02, ...,
         3.0044874e-04, 3.0036236e-04, 1.4643306e-06],
        [2.3392391e-01, 2.9374105e-01, 9.3837559e-02, ...,
         3.0044874e-04, 3.0036236e-04, 0.0000000e+00]]], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:50.547000
    zwu      (z) float64 1.5 3.0 6.0 9.0 12.0 ... 567.0 570.0 573.0 576.0 579.0
  * z        (z) float64 1.5 3.0 6.0 9.0 12.0 ... 567.0 570.0 573.0 576.0 579.0

In [88]:
ustar_bar = ustar.isel(z=0).mean()

In [89]:
ustar_bar

<xarray.DataArray ()>
array(0.23392392)
Coordinates:
    zwu      float64 1.5
    z        float64 1.5

In [83]:
ds_xy["us*_xy"].mean()

<xarray.DataArray 'us*_xy' ()>
array(0.25189131, dtype=float32)

In [272]:
ds_lad

<xarray.Dataset>
Dimensions:  (x: 192, y: 432, zlad: 6)
Coordinates:
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
  * zlad     (zlad) int32 0 1 4 7 10 13
Data variables:
    lad      (zlad, y, x) float64 dask.array<chunksize=(6, 432, 192), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.7
    origin_lat:      40.16339309801363
    origin_lon:      -83.16731841749935
    origin_time:     2022-02-27 12:00:00 +00
    origin_x:        315432.564
    origin_y:        4448144.558
    origin_z:        275.0
    rotation_angle:  0.0

In [299]:
DR = (ubar.isel(y=y_domain) * ds.s.isel(y=y_domain) * ds_lad.lad.isel(y=y_domain)).compute()

<xarray.DataArray (time: 10, z: 4, y: 216, x: 191)>
array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 3.15568049e-02, 2.80204944e-02, ...,
          2.75981633e-02, 2.96342856e-02, 3.22920548e-02],
         [0.00000000e+00, 1.85876214e-02, 1.86781800e-02, ...,
          1.89125432e-02, 1.90676492e-02, 1.74037601e-02]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
...
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 6.17549136e-02, 2.21250701e-02, ...,
          1.04452037e-02, 2.48641812e-02, 1.39792557e-01],
         [0.00000000e+00, 2.02564239e-03, 7.11602362e-04, ...,
          1.80500225e-03, 6.02498310e-04, 1.60397549e-04]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 2.31699483e-01, 7.70043582e-02, ...,
          3.09521822e-02, 7.40247796e-02, 2.41035339e-01],
         [0.00000000e+00, 2.81722276e-02, 5.79994243e-09, ...,
          1.37116482e-03, 1.00913538e-12, 2.90156350e-02]]]])
Coordinates:
  * z        (z) float64 4.5 7.5 10.5 13.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:00:50.064000
  * y        (y) float64 646.5 649.5 652.5 ... 1.286e+03 1.288e+03 1.292e+03
    xu       (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
    yv       (y) float64 646.5 649.5 652.5 ... 1.286e+03 1.288e+03 1.292e+03
    zw_3d    (z) float64 4.5 7.5 10.5 13.5

In [304]:
DRVsTime = DR.mean(dim=["x", "y"])

In [327]:
lai_vals = ds_lad.lad.sum(dim="z").values
lai = lai_vals[lai_vals != 0].mean()

In [172]:
ubar_z_scalar = ubar.isel(y=y_domain, z=z_scalar).mean(skipna=True)
scalar_gradient = ds.isel(y=y_domain, z=z_scalar).s.mean(skipna=True)

In [336]:

ustar_bar**2

<xarray.DataArray ()>
array(1.7688297e-05, dtype=float32)

In [342]:
rah = ubar_z_scalar/(ustar_bar**2) + 6.2 / (ustar_bar**(2/3))
rah

<xarray.DataArray ()>
array(351.57892059)
Coordinates:
    z        float64 21.0

In [310]:
Depos = DRVsTime.mean(skipna=True)

In [343]:
ustar_bar * lai * (scalar_gradient / Depos - rah - 1/(ustar_bar * lai))

<xarray.DataArray ()>
array(-123.58734897)
Coordinates:
    z        float64 21.0

In [ ]:
LAI*0.5*Ustar*ScalarGrad/Depos-LAI*0.5*rah*Ustar-1

In [ ]:
Depos = nanmean(nanmean(DRVsTime(:,1:4),2),1);
Alpha = LAI*0.5*Ustar*ScalarGrad/Depos-LAI*0.5*rah*Ustar-1;

In [5]:
# try:
#     ds.chunks
# except ValueError:
#     ds = ds.unify_chunks()
# try:
#     ds_lad.chunks
# except ValueError:
#     ds_lad = ds_lad.unify_chunks()


In [58]:
ds2 = xr.Dataset(
    data_vars={
        "a": ("x", [5, 7, 4]),
        "b": (
            ("x", "y"),
            [[1, 4, 2, 9], [2, 7, 6, 12], [6, 10, 5, 8]],
        ),
    },
    coords={"x": [0, 3, 6], "y": [0, 3, 6, 9]},
)
ds2.a

<xarray.DataArray 'a' (x: 3)>
array([5, 7, 4])
Coordinates:
  * x        (x) int64 0 3 6

In [73]:
ds2.interp(x=[1.5, 4.5], y=[1.5, 4.5, 7.5]).b

<xarray.Dataset>
Dimensions:  (time: 10, zu_3d: 192, zw_3d: 193, x: 191, xu: 192, y: 431,
              yv: 432, zpc_3d: 6)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:00:50.064000
  * zu_3d    (zu_3d) float64 1.5 4.5 7.5 10.5 13.5 ... 565.5 568.5 571.5 574.5
  * zw_3d    (zw_3d) float64 3.0 6.0 9.0 12.0 15.0 ... 570.0 573.0 576.0 579.0
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 559.5 562.5 565.5 568.5 571.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.286e+03 1.288e+03 1.292e+03
  * yv       (yv) float64 0.0 3.0 6.0 9.0 ... 1.287e+03 1.29e+03 1.293e+03
  * zpc_3d   (zpc_3d) float64 0.0 1.5 4.5 7.5 10.5 13.5
Data variables: (12/24)
    E_UTM    (x) float64 ...
    N_UTM    (y) float64 ...
    Eu_UTM   (xu) float64 ...
    Nu_UTM   (y) float64 ...
    Ev_UTM   (x) float64 ...
    Nv_UTM   (yv) float64 ...
    ...       ...
    uu       (time, zu_3d, y, xu) float32 ...
    vv       (time, zu_3d, yv, x) float32 ...
    ww       (time, zw_3d, y, x) float32 ...
    wu       (time, zu_3d, y, x) float32 ...
    wv       (time, zu_3d, y, x) float32 ...
    e        (time, zu_3d, y, x) float32 ...
Attributes: (12/27)
    title:           PALM 6.0  Rev: 4359M  run: 20220308T183847.00  host: def...
    Conventions:     CF-1.7
    creation_time:   2022-03-14 10:13:18 -04
    data_content:    3d
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    source:          PALM 6.0 Rev: 4359M
    references:      
    keywords:        
    licence:         
    comment:         
    VAR_LIST:        ;theta;v;w;u;s;uu;vv;ww;wu;wv;e;

<xarray.DataArray 'theta' (time: 10, zu_3d: 194, y: 432, x: 192)>
[160911360 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:00:50.064000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      K
    long_name:  theta

In [6]:
(ds_ts.E * 2) ** (1/2)

<xarray.DataArray 'E' (time: 91)>
array([3.9767377, 3.9747574, 3.9732218, 3.971906 , 3.970735 , 3.969675 ,
       3.968696 , 3.967786 , 3.9669385, 3.966143 , 3.9653955, 3.9646978,
       3.9640458, 3.9634352, 3.9628706, 3.9623547, 3.9618852, 3.9614623,
       3.9610796, 3.9607313, 3.9604115, 3.9601116, 3.9598222, 3.959537 ,
       3.9592497, 3.9589555, 3.958656 , 3.958352 , 3.9580472, 3.957745 ,
       3.9574502, 3.9571671, 3.9568968, 3.9566395, 3.956394 , 3.9561584,
       3.9559326, 3.9557152, 3.9555063, 3.9553056, 3.9551132, 3.9549296,
       3.9547546, 3.9545872, 3.9544263, 3.9542713, 3.9541206, 3.9539733,
       3.9538271, 3.9536812, 3.9535346, 3.9533856, 3.9532335, 3.9530785,
       3.952921 , 3.9527612, 3.9526012, 3.952441 , 3.9522817, 3.9521239,
       3.9519684, 3.9518154, 3.9516659, 3.9515195, 3.9513772, 3.9512389,
       3.9511049, 3.950975 , 3.950849 , 3.9507267, 3.950607 , 3.9504902,
       3.9503756, 3.950262 , 3.9501498, 3.9500372, 3.9499242, 3.9498096,
       3.9496925, 3.9495733, 3.949451 , 3.9493263, 3.9491997, 3.949072 ,
       3.9489436, 3.9488168, 3.9486914, 3.9485688, 3.9484491, 3.9483316,
       3.9482179], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:00.635000 ... 00:00:50.064000

In [8]:
ds_3d_av.wu - ds_3d_av.u * ds_3d_av.w

MemoryError: Unable to allocate 2.18 TiB for an array with shape (1, 194, 432, 192, 194, 192) and data type float32

In [12]:
ds_3d_av.u.zu_3d

<xarray.DataArray 'zu_3d' (zu_3d: 194)>
array([  0. ,   1.5,   4.5,   7.5,  10.5,  13.5,  16.5,  19.5,  22.5,  25.5,
        28.5,  31.5,  34.5,  37.5,  40.5,  43.5,  46.5,  49.5,  52.5,  55.5,
        58.5,  61.5,  64.5,  67.5,  70.5,  73.5,  76.5,  79.5,  82.5,  85.5,
        88.5,  91.5,  94.5,  97.5, 100.5, 103.5, 106.5, 109.5, 112.5, 115.5,
       118.5, 121.5, 124.5, 127.5, 130.5, 133.5, 136.5, 139.5, 142.5, 145.5,
       148.5, 151.5, 154.5, 157.5, 160.5, 163.5, 166.5, 169.5, 172.5, 175.5,
       178.5, 181.5, 184.5, 187.5, 190.5, 193.5, 196.5, 199.5, 202.5, 205.5,
       208.5, 211.5, 214.5, 217.5, 220.5, 223.5, 226.5, 229.5, 232.5, 235.5,
       238.5, 241.5, 244.5, 247.5, 250.5, 253.5, 256.5, 259.5, 262.5, 265.5,
       268.5, 271.5, 274.5, 277.5, 280.5, 283.5, 286.5, 289.5, 292.5, 295.5,
       298.5, 301.5, 304.5, 307.5, 310.5, 313.5, 316.5, 319.5, 322.5, 325.5,
       328.5, 331.5, 334.5, 337.5, 340.5, 343.5, 346.5, 349.5, 352.5, 355.5,
       358.5, 361.5, 364.5, 367.5, 370.5, 373.5, 376.5, 379.5, 382.5, 385.5,
       388.5, 391.5, 394.5, 397.5, 400.5, 403.5, 406.5, 409.5, 412.5, 415.5,
       418.5, 421.5, 424.5, 427.5, 430.5, 433.5, 436.5, 439.5, 442.5, 445.5,
       448.5, 451.5, 454.5, 457.5, 460.5, 463.5, 466.5, 469.5, 472.5, 475.5,
       478.5, 481.5, 484.5, 487.5, 490.5, 493.5, 496.5, 499.5, 502.5, 505.5,
       508.5, 511.5, 514.5, 517.5, 520.5, 523.5, 526.5, 529.5, 532.5, 535.5,
       538.5, 541.5, 544.5, 547.5, 550.5, 553.5, 556.5, 559.5, 562.5, 565.5,
       568.5, 571.5, 574.5, 577.5])
Coordinates:
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
Attributes:
    units:    meters

In [11]:
ds_3d_av.w.zw_3d

<xarray.DataArray 'zw_3d' (zw_3d: 194)>
array([  0.,   3.,   6.,   9.,  12.,  15.,  18.,  21.,  24.,  27.,  30.,  33.,
        36.,  39.,  42.,  45.,  48.,  51.,  54.,  57.,  60.,  63.,  66.,  69.,
        72.,  75.,  78.,  81.,  84.,  87.,  90.,  93.,  96.,  99., 102., 105.,
       108., 111., 114., 117., 120., 123., 126., 129., 132., 135., 138., 141.,
       144., 147., 150., 153., 156., 159., 162., 165., 168., 171., 174., 177.,
       180., 183., 186., 189., 192., 195., 198., 201., 204., 207., 210., 213.,
       216., 219., 222., 225., 228., 231., 234., 237., 240., 243., 246., 249.,
       252., 255., 258., 261., 264., 267., 270., 273., 276., 279., 282., 285.,
       288., 291., 294., 297., 300., 303., 306., 309., 312., 315., 318., 321.,
       324., 327., 330., 333., 336., 339., 342., 345., 348., 351., 354., 357.,
       360., 363., 366., 369., 372., 375., 378., 381., 384., 387., 390., 393.,
       396., 399., 402., 405., 408., 411., 414., 417., 420., 423., 426., 429.,
       432., 435., 438., 441., 444., 447., 450., 453., 456., 459., 462., 465.,
       468., 471., 474., 477., 480., 483., 486., 489., 492., 495., 498., 501.,
       504., 507., 510., 513., 516., 519., 522., 525., 528., 531., 534., 537.,
       540., 543., 546., 549., 552., 555., 558., 561., 564., 567., 570., 573.,
       576., 579.])
Coordinates:
  * zw_3d    (zw_3d) float64 0.0 3.0 6.0 9.0 12.0 ... 570.0 573.0 576.0 579.0
Attributes:
    units:    meters

In [42]:
ds_3d_av.wu

<xarray.DataArray 'wu' (time: 1, zu_3d: 194, y: 432, x: 192)>
[16091136 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:50.064000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      m2/s2
    long_name:  wu

In [38]:
ds_pr.wu

<xarray.DataArray 'wu' (time: 1, zwu: 194)>
array([[ 3.708458e-06, -2.404072e-06, -8.293280e-06,  1.740860e-06,
         4.843925e-05,  5.478504e-05,  6.241943e-05,  3.505694e-05,
        -1.249914e-06, -2.664011e-05, -5.045217e-06, -1.408841e-05,
         1.031819e-05,  1.122223e-05,  1.098622e-05, -7.514377e-06,
         1.752835e-05,  5.424442e-05,  6.094183e-05,  3.813928e-05,
        -4.615026e-06, -1.727041e-05, -2.045674e-05, -4.750245e-06,
        -6.689460e-06, -5.110061e-06,  4.651324e-05,  5.002182e-05,
         5.265976e-05,  3.846427e-05,  2.807785e-05,  2.411686e-05,
         4.469390e-07,  1.912070e-06,  2.441524e-05,  3.584686e-06,
        -2.688499e-06, -9.777516e-06, -1.518766e-05,  1.670871e-06,
         1.119985e-05,  1.904737e-05,  2.149273e-05,  9.885601e-06,
        -1.771408e-06,  5.000838e-06, -8.186413e-06, -2.683495e-05,
        -1.019449e-05,  3.747525e-05,  2.789108e-05,  1.276502e-05,
         1.632423e-05, -1.196531e-05,  8.198333e-06,  8.986671e-06,
        -2.979140e-05, -1.818442e-05,  2.084425e-05,  1.494805e-05,
         2.419399e-05,  3.397439e-05,  1.048540e-05,  1.563866e-05,
         1.302108e-05,  2.631690e-06, -3.119811e-07, -6.455523e-08,
         2.874433e-08,  6.619253e-09, -1.433693e-09, -1.873131e-10,
         7.718966e-11,  8.082420e-13, -4.355497e-12, -1.093771e-13,
         5.740942e-13,  9.784327e-13,  1.360738e-12,  1.637130e-12,
         1.837987e-12,  1.985690e-12,  2.093423e-12,  2.170865e-12,
         2.225126e-12,  2.261446e-12,  2.283752e-12,  2.295025e-12,
         2.297568e-12,  2.293175e-12,  2.283270e-12,  2.268990e-12,
         2.251258e-12,  2.230824e-12,  2.208308e-12,  2.184219e-12,
         2.158984e-12,  2.132958e-12,  2.106435e-12,  2.079663e-12,
         2.052850e-12,  2.026167e-12,  1.999756e-12,  1.973736e-12,
         1.948204e-12,  1.923237e-12,  1.898897e-12,  1.875234e-12,
         1.852285e-12,  1.830076e-12,  1.808628e-12,  1.787951e-12,
         1.768051e-12,  1.748927e-12,  1.730574e-12,  1.712985e-12,
         1.696148e-12,  1.680048e-12,  1.664670e-12,  1.649995e-12,
         1.636005e-12,  1.622678e-12,  1.609994e-12,  1.597931e-12,
         1.586467e-12,  1.575581e-12,  1.565247e-12,  1.555441e-12,
         1.546213e-12,  1.537375e-12,  1.527515e-12,  1.524091e-12,
         1.515126e-12,  1.469116e-12,  1.397764e-12,  1.329418e-12,
         1.266731e-12,  1.207164e-12,  1.151022e-12,  1.098102e-12,
         1.048240e-12,  1.001279e-12,  9.570679e-13,  9.154624e-13,
         8.763250e-13,  8.395241e-13,  8.049342e-13,  7.724356e-13,
         7.419146e-13,  7.132626e-13,  6.863765e-13,  6.611583e-13,
         6.375150e-13,  6.153581e-13,  5.946038e-13,  5.751727e-13,
         5.569892e-13,  5.399820e-13,  5.240834e-13,  5.092295e-13,
         4.953595e-13,  4.824162e-13,  4.703453e-13,  4.590955e-13,
         4.486185e-13,  4.388683e-13,  4.298017e-13,  4.213779e-13,
         4.135582e-13,  4.063062e-13,  3.995874e-13,  3.933694e-13,
         3.876215e-13,  3.823148e-13,  3.774219e-13,  3.729169e-13,
         3.687756e-13,  3.649750e-13,  3.614933e-13,  3.583100e-13,
         3.554057e-13,  3.527622e-13,  3.503622e-13,  3.481892e-13,
         3.462278e-13,  3.444634e-13,  3.428821e-13,  3.414707e-13,
         3.402163e-13,  3.391081e-13,  3.381546e-13,  3.371813e-13,
         3.368448e-13,  0.000000e+00]], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:50.064000
  * zwu      (zwu) float64 1.5 3.0 6.0 9.0 12.0 ... 570.0 573.0 576.0 579.0
Attributes:
    units:      m2/s2
    long_name:  wu

In [31]:
ds_xy

<xarray.Dataset>
Dimensions:   (time: 1, zu_xy: 2, zw_xy: 2, zu1_xy: 1, x: 192, xu: 192, y: 432,
               yv: 432)
Coordinates:
  * time      (time) timedelta64[ns] 00:00:05.179000
  * zu_xy     (zu_xy) float64 148.5 298.5
  * zw_xy     (zw_xy) float64 150.0 300.0
  * zu1_xy    (zu1_xy) float64 1.5
  * x         (x) float64 1.5 4.5 7.5 10.5 13.5 ... 565.5 568.5 571.5 574.5
  * xu        (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 564.0 567.0 570.0 573.0
  * y         (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
  * yv        (yv) float64 0.0 3.0 6.0 9.0 ... 1.287e+03 1.29e+03 1.293e+03
Data variables: (12/15)
    ind_z_xy  (zu_xy) float64 ...
    E_UTM     (x) float64 ...
    N_UTM     (y) float64 ...
    Eu_UTM    (xu) float64 ...
    Nu_UTM    (y) float64 ...
    Ev_UTM    (x) float64 ...
    ...        ...
    lonu      (y, xu) float64 ...
    latu      (y, xu) float64 ...
    lonv      (yv, x) float64 ...
    latv      (yv, x) float64 ...
    crs       int32 ...
    us*_xy    (time, zu1_xy, y, x) float32 ...
Attributes: (12/27)
    title:           PALM 6.0  Rev: 4359  run: 20220308T183847.00  host: defa...
    Conventions:     CF-1.7
    creation_time:   2022-03-13 16:01:20 -04
    data_content:    xy
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    source:          PALM 6.0 Rev: 4359
    references:      
    keywords:        
    licence:         
    comment:         
    VAR_LIST:        ;us*_xy;

In [39]:
ds_ts

<xarray.Dataset>
Dimensions:      (time: 91)
Coordinates:
  * time         (time) timedelta64[ns] 00:00:00.635000 ... 00:00:50.064000
Data variables: (12/25)
    E            (time) float32 ...
    E*           (time) float32 ...
    dt           (time) float32 ...
    us*          (time) float32 ...
    th*          (time) float32 ...
    umax         (time) float32 ...
    ...           ...
    w"v"0        (time) float32 ...
    w"q"0        (time) float32 ...
    ol           (time) float32 ...
    q*           (time) float32 ...
    w"s"         (time) float32 ...
    s*           (time) float32 ...
Attributes: (12/27)
    title:           PALM 6.0  Rev: 4359  run: 20220308T183847.00  host: defa...
    Conventions:     CF-1.7
    creation_time:   2022-03-13 16:01:20 -04
    data_content:    podsts
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    source:          PALM 6.0 Rev: 4359
    references:      
    keywords:        
    licence:         
    comment:         
    VAR_LIST:        ;E;E*;dt;us*;th*;umax;vmax;wmax;div_new;div_old;zi_wthet...

In [43]:
TempSpacMeanU, TempSpacMeanV, TempSpacMeanW = calc_temp_spac_means(ds)

In [14]:
my_uw = ds.w.values * ds.u.values

In [15]:
my_uw

array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[-4.21243487e-03,  2.80405842e-02,  9.05366521e-03, ...,
           2.55464041e-03, -4.01457585e-03, -2.41390504e-02],
         [-2.12163781e-03,  7.03842640e-02,  4.27138172e-02, ...,
          -2.57333666e

In [16]:
ds.wu.values

array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[-4.21243487e-03,  2.80405842e-02,  9.05366428e-03, ...,
           2.55464041e-03, -4.01457585e-03, -2.41390504e-02],
         [-2.12163758e-03,  7.03842640e-02,  4.27138209e-02, ...,
          -2.57333685e

In [ ]:
(wt(:,CanHomY,:,:) - TempSpacCanMeanW).*(ut(:,CanHomY,:,:) - TempSpacCanMeanU)

In [ ]:
ds_xy["us*_xy"]

In [6]:
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

a = np.array([[1, 2],
              [3, 4]])
b = np.array([[5, 6],
              [7, 8]])

In [40]:
np.((a**2 + b**2)**(1/2))

27.98361984979261

In [21]:
%timeit (ds.uu.sum(skipna=True) + ds.vv.sum(skipna=True) + ds.ww.sum(skipna=True))

1.75 s ± 56.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
ds.uu

<xarray.DataArray 'uu' (time: 20, zu_3d: 194, y: 432, xu: 192)>
dask.array<open_dataset-9d5e0dcffbe174793ec03ae74f7a3d6auu, shape=(20, 194, 432, 192), dtype=float32, chunksize=(10, 97, 216, 96), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:01:40.066000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      m2/s2
    long_name:  uu

In [27]:
(ds.u**2).load()

<xarray.DataArray 'u' (time: 20, zu_3d: 194, y: 432, xu: 192)>
array([[[[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]],

        [[3.02269385e-04, 3.83085664e-03, 4.08186635e-04, ...,
          3.07246846e-05, 6.47973648e-05, 1.06724445e-02],
         [3.84246159e-05, 1.41614564e-02, 4.97492822e-03, ...,
          1.74774614e-03, 4.25005332e-03, 1.83803216e-02],
         [5.76714119e-05, 6.99179154e-03, 4.25800309e-03, ...,
          4.33023972e-03, 4.46787430e-03, 6.17682701e-03],
...
          1.29040971e-11, 1.11548800e-11, 9.50852678e-12],
         [7.30895639e-12, 5.95128634e-12, 4.72025110e-12, ...,
          1.20743918e-11, 1.03777846e-11, 8.78679508e-12],
         [6.68836340e-12, 5.38982357e-12, 4.21977774e-12, ...,
          1.12901164e-11, 9.64553784e-12, 8.10919040e-12]],

        [[5.83084405e-12, 4.61818951e-12, 3.53690406e-12, ...,
          1.01959274e-11, 8.62638520e-12, 7.16904314e-12],
         [5.30859514e-12, 4.15263310e-12, 3.12960378e-12, ...,
          9.51543965e-12, 7.99643338e-12, 6.59191582e-12],
         [4.82308377e-12, 3.72269490e-12, 2.75668616e-12, ...,
          8.87431535e-12, 7.40509650e-12, 6.05252567e-12],
         ...,
         [7.63664392e-12, 6.24755803e-12, 4.98442303e-12, ...,
          1.24908443e-11, 1.07656939e-11, 9.14507272e-12],
         [6.99297868e-12, 5.66372166e-12, 4.46234759e-12, ...,
          1.16818846e-11, 1.00092183e-11, 8.44377606e-12],
         [6.39167885e-12, 5.12124674e-12, 3.98050256e-12, ...,
          1.09174969e-11, 9.29671704e-12, 7.78571912e-12]]]],
      dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:01:40.066000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03

In [22]:
%timeit (ds.u**2)

1.62 s ± 88.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
ds_xy["us*_xy"]

<xarray.DataArray 'us*_xy' (time: 1, zu1_xy: 1, y: 432, x: 192)>
array([[[[0.386791, ..., 0.383674],
         ...,
         [0.455789, ..., 0.449921]]]], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000
  * zu1_xy   (zu1_xy) float64 1.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      m/s
    long_name:  us*_xy

In [10]:
ds_ts

<xarray.Dataset>
Dimensions:      (time: 20)
Coordinates:
  * time         (time) timedelta64[ns] 00:00:05.179000 ... 00:01:40.066000
Data variables: (12/25)
    E            (time) float32 7.868 7.847 7.835 7.826 ... 7.765 7.761 7.757
    E*           (time) float32 0.009694 0.01585 0.01892 ... 0.03515 0.03565
    dt           (time) float32 0.554 0.568 0.555 0.551 ... 0.519 0.516 0.508
    us*          (time) float32 0.155 0.2018 0.2048 ... 0.204 0.2053 0.2064
    th*          (time) float32 -0.1892 -0.1161 -0.0807 ... -0.06465 -0.06183
    umax         (time) float32 0.5291 0.5369 -0.7722 ... 1.992 2.255 2.45
    ...           ...
    w"v"0        (time) float32 -0.05257 -0.08168 -0.06836 ... -0.04203 -0.04206
    w"q"0        (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ol           (time) float32 -2.146e+04 -2.352e+04 ... -3.176e+03 -3.276e+03
    q*           (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    w"s"         (time) float32 -1.028e-17 8.493e-14 ... 1.225e-05 4.465e-05
    s*           (time) float32 -19.63 -12.2 -8.839 ... -8.144 -8.027 -7.779
Attributes: (12/27)
    title:           PALM 6.0  Rev: 4359  run: 20220308T183847.00  host: defa...
    Conventions:     CF-1.7
    creation_time:   2022-03-12 20:42:11 -05
    data_content:    podsts
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    source:          PALM 6.0 Rev: 4359
    references:      
    keywords:        
    licence:         
    comment:         
    VAR_LIST:        ;E;E*;dt;us*;th*;umax;vmax;wmax;div_new;div_old;zi_wthet...

In [73]:
ds_lad.chunks

Frozen({'zlad': (6,), 'y': (432,), 'x': (192,)})

In [74]:
ds.chunks

Frozen({'x': (64, 64, 64), 'y': (144, 144, 144), 'xu': (64, 64, 64), 'yv': (144, 144, 144), 'time': (24, 24, 24), 'zu_3d': (97, 97), 'zw_3d': (97, 97)})

In [89]:
s = ds.isel(x=[1,2,3,4,5], y=[1,2,3,4,5], time=[1,2,3,4,5], zu_3d=[1,2,3,4,5], xu=[1,2,3,4,5], zw_3d=[1,2,3,4,5], yv=[1,2,3,4,5])

In [98]:
((s.u.values**2 + s.v.values**2 +s.w.values**2)**(1/2)).mean()

1.4358897

In [67]:
ds.u

<xarray.DataArray 'u' (time: 72, zu_3d: 194, y: 432, xu: 192)>
dask.array<open_dataset-b1a2d31fd1c134903f89ae5a2909490eu, shape=(72, 194, 432, 192), dtype=float32, chunksize=(24, 97, 144, 64), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      m/s
    long_name:  u

In [69]:
%time ((ds.v[...,1:,0:-2] + ds.v[...,0:-2,0:-2])/2).compute()

/Users/madelinescyphers/.conda/envs/palm/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/Users/madelinescyphers/.conda/envs/palm/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self

CPU times: user 24.9 s, sys: 42.6 s, total: 1min 7s
Wall time: 47.2 s


<xarray.DataArray 'v' (time: 72, zu_3d: 194, yv: 429, x: 190)>
array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[ 9.10635769e-01,  9.10324454e-01,  9.53532100e-01, ...,
           1.01259840e+00,  9.74060833e-01,  9.27985430e-01],
         [ 4.43166941e-01,  3.67338330e-01,  3.59824628e-01, ...,
           3.92852902e-01,  3.56822342e-01,  3.35830152e-01],
         [-1.47885541e-02, -1.09273665e-01, -1.59744218e-01, ...,
          -1.51178211e-01, -1.88755289e-01, -1.79650769e-01],
...
           4.02614450e+00,  4.02614450e+00,  4.02614450e+00],
         [ 4.02584171e+00,  4.02584219e+00,  4.02584219e+00, ...,
           4.02584124e+00,  4.02584124e+00,  4.02584171e+00],
         [ 4.02553272e+00,  4.02553272e+00,  4.02553272e+00, ...,
           4.02553225e+00,  4.02553225e+00,  4.02553225e+00]],

        [[ 4.02430725e+00,  4.02430773e+00,  4.02430773e+00, ...,
           4.02430725e+00,  4.02430725e+00,  4.02430725e+00],
         [ 4.02397060e+00,  4.02397060e+00,  4.02397060e+00, ...,
           4.02397013e+00,  4.02397013e+00,  4.02397013e+00],
         [ 4.02362823e+00,  4.02362823e+00,  4.02362823e+00, ...,
           4.02362776e+00,  4.02362776e+00,  4.02362776e+00],
         ...,
         [ 4.02620935e+00,  4.02620935e+00,  4.02620935e+00, ...,
           4.02620840e+00,  4.02620888e+00,  4.02620888e+00],
         [ 4.02590704e+00,  4.02590704e+00,  4.02590704e+00, ...,
           4.02590656e+00,  4.02590656e+00,  4.02590656e+00],
         [ 4.02559853e+00,  4.02559900e+00,  4.02559900e+00, ...,
           4.02559805e+00,  4.02559805e+00,  4.02559853e+00]]]],
      dtype=float32)
Coordinates:
  * yv       (yv) float64 3.0 6.0 9.0 12.0 ... 1.281e+03 1.284e+03 1.287e+03
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 556.5 559.5 562.5 565.5 568.5

In [ ]:
    vt = (vt(1:end-1,2:end,:,:) + vt(1:end-1,1:end-1,:,:))/2;
    
    ut = ncread(OutputFile,'u',[1 1 1 t],[96 432 194 12]);
    ut(:,:,1,:) = [];
    ut = (ut(2:end,1:end-1,:,:) + ut(1:end-1,1:end-1,:,:))/2;
    
    wt = ncread(OutputFile,'w',[1 1 1 t],[96 432 194 12]);
    wt(:,:,1,:) = 0;
    wt = (wt(1:end-1,1:end-1,2:end,:) + wt(1:end-1,1:end-1,1:end-1,:))/2;

In [9]:
# ds_lad.isel(zlad=1).lad.values

In [ ]:
# ds

In [ ]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)
# print(time.time() - s)
# # ubar_total = np.sqrt(np.square(ds.u.values) + np.square(ds.v.values) + np.square(ds.w.values))

In [5]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# # ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

# ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
# print(time.time() - s)

23.01164722442627


In [ ]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# # ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

# ubar_total = np.sqrt(np.square(sub_ds.u) + np.square(sub_ds.v) + np.square(sub_ds.w))
# # ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
# print(time.time() - s)

In [53]:
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar,)# x=1, time=0, xu=1)

In [54]:
s = time.time()
sub_ds.u**2
print(time.time() - s)
sub_ds


0.0021800994873046875


<xarray.Dataset>
Dimensions:  (time: 72, zu_3d: 50, zw_3d: 50, x: 192, xu: 192, y: 216, yv: 216, zpc_3d: 6)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 148.5 151.5 154.5 157.5 ... 286.5 289.5 292.5 295.5
  * zw_3d    (zw_3d) float64 150.0 153.0 156.0 159.0 ... 288.0 291.0 294.0 297.0
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
  * yv       (yv) float64 648.0 651.0 654.0 ... 1.287e+03 1.29e+03 1.293e+03
  * zpc_3d   (zpc_3d) float64 0.0 1.5 4.5 7.5 10.5 13.5
Data variables: (12/18)
    E_UTM    (x) float64 dask.array<chunksize=(64,), meta=np.ndarray>
    N_UTM    (y) float64 dask.array<chunksize=(72,), meta=np.ndarray>
    Eu_UTM   (xu) float64 dask.array<chunksize=(64,), meta=np.ndarray>
    Nu_UTM   (y) float64 dask.array<chunksize=(72,), meta=np.ndarray>
    Ev_UTM   (x) float64 dask.array<chunksize=(64,), meta=np.ndarray>
    Nv_UTM   (yv) float64 dask.array<chunksize=(72,), meta=np.ndarray>
    ...       ...
    crs      int32 ...
    theta    (time, zu_3d, y, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    v        (time, zu_3d, yv, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    w        (time, zw_3d, y, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    u        (time, zu_3d, y, xu) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    s        (time, zu_3d, y, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
Attributes: (12/27)
    title:           PALM 6.0  Rev: 4359  run: 20220308T183847.00  host: defa...
    Conventions:     CF-1.7
    creation_time:   2022-03-08 18:41:16 -05
    data_content:    3d
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    source:          PALM 6.0 Rev: 4359
    references:      
    keywords:        
    licence:         
    comment:         
    VAR_LIST:        ;theta;v;w;u;s;

In [55]:
sub_ds.u

<xarray.DataArray 'u' (time: 72, zu_3d: 50, y: 216, xu: 192)>
dask.array<getitem, shape=(72, 50, 216, 192), dtype=float32, chunksize=(24, 47, 144, 64), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 148.5 151.5 154.5 157.5 ... 286.5 289.5 292.5 295.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      m/s
    long_name:  u

In [56]:
sub_ds.v.zu_3d

<xarray.DataArray 'zu_3d' (zu_3d: 50)>
array([148.5, 151.5, 154.5, 157.5, 160.5, 163.5, 166.5, 169.5, 172.5, 175.5,
       178.5, 181.5, 184.5, 187.5, 190.5, 193.5, 196.5, 199.5, 202.5, 205.5,
       208.5, 211.5, 214.5, 217.5, 220.5, 223.5, 226.5, 229.5, 232.5, 235.5,
       238.5, 241.5, 244.5, 247.5, 250.5, 253.5, 256.5, 259.5, 262.5, 265.5,
       268.5, 271.5, 274.5, 277.5, 280.5, 283.5, 286.5, 289.5, 292.5, 295.5])
Coordinates:
  * zu_3d    (zu_3d) float64 148.5 151.5 154.5 157.5 ... 286.5 289.5 292.5 295.5
Attributes:
    units:    meters

In [57]:
sub_ds.w.zw_3d

<xarray.DataArray 'zw_3d' (zw_3d: 50)>
array([150., 153., 156., 159., 162., 165., 168., 171., 174., 177., 180., 183.,
       186., 189., 192., 195., 198., 201., 204., 207., 210., 213., 216., 219.,
       222., 225., 228., 231., 234., 237., 240., 243., 246., 249., 252., 255.,
       258., 261., 264., 267., 270., 273., 276., 279., 282., 285., 288., 291.,
       294., 297.])
Coordinates:
  * zw_3d    (zw_3d) float64 150.0 153.0 156.0 159.0 ... 288.0 291.0 294.0 297.0
Attributes:
    units:    meters

In [24]:
temp = sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2

In [23]:
%time temp.load()

CPU times: user 3.78 s, sys: 10.2 s, total: 14 s
Wall time: 8.65 s


<xarray.DataArray (zu_3d: 50, y: 216, yv: 216, zw_3d: 50)>
array([[[[15.72673  , 15.726302 , 15.726774 , ..., 15.726437 ,
          15.726431 , 15.726424 ],
         [15.627536 , 15.627108 , 15.62758  , ..., 15.627242 ,
          15.627236 , 15.62723  ],
         [15.68143  , 15.681002 , 15.681474 , ..., 15.681136 ,
          15.68113  , 15.681124 ],
         ...,
         [16.54239  , 16.541962 , 16.542435 , ..., 16.542097 ,
          16.54209  , 16.542084 ],
         [16.62572  , 16.62529  , 16.625763 , ..., 16.625425 ,
          16.625418 , 16.625412 ],
         [16.384722 , 16.384293 , 16.384766 , ..., 16.384428 ,
          16.38442  , 16.384415 ]],

        [[15.72759  , 15.726189 , 15.726829 , ..., 15.726372 ,
          15.726365 , 15.726358 ],
         [15.628395 , 15.626994 , 15.627634 , ..., 15.627177 ,
          15.627171 , 15.627164 ],
         [15.682289 , 15.680888 , 15.681528 , ..., 15.681071 ,
          15.681065 , 15.681058 ],
...
         [16.017162 , 16.017141 , 16.013308 , ..., 16.012814 ,
          16.012808 , 16.012802 ],
         [16.015526 , 16.015505 , 16.011671 , ..., 16.011177 ,
          16.011171 , 16.011166 ],
         [16.01389  , 16.013868 , 16.010035 , ..., 16.00954  ,
          16.009535 , 16.00953  ]],

        [[15.990481 , 15.989479 , 15.986883 , ..., 15.986002 ,
          15.985996 , 15.985991 ],
         [15.992192 , 15.99119  , 15.988594 , ..., 15.987713 ,
          15.987707 , 15.987701 ],
         [15.99391  , 15.992908 , 15.990312 , ..., 15.98943  ,
          15.989425 , 15.989419 ],
         ...,
         [16.017294 , 16.01629  , 16.013697 , ..., 16.012815 ,
          16.01281  , 16.012804 ],
         [16.015657 , 16.014654 , 16.01206  , ..., 16.011179 ,
          16.011173 , 16.011168 ],
         [16.01402  , 16.013018 , 16.010424 , ..., 16.009542 ,
          16.009537 , 16.009531 ]]]], dtype=float32)
Coordinates:
    time     timedelta64[ns] 00:00:05.179000
    x        float64 4.5
    xu       float64 3.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 151.5 154.5 157.5 160.5 ... 289.5 292.5 295.5 298.5
  * yv       (yv) float64 648.0 651.0 654.0 ... 1.287e+03 1.29e+03 1.293e+03
    zw_3d_   (zw_3d) float64 153.0 156.0 159.0 162.0 ... 291.0 294.0 297.0 300.0
Dimensions without coordinates: zu_3d, zw_3d

In [26]:
temp.shape

(50, 216, 216, 50)

In [25]:
%time temp.compute()

CPU times: user 3.4 s, sys: 7.94 s, total: 11.3 s
Wall time: 5.7 s


<xarray.DataArray (zu_3d: 50, y: 216, yv: 216, zw_3d: 50)>
array([[[[15.72673  , 15.726302 , 15.726774 , ..., 15.726437 ,
          15.726431 , 15.726424 ],
         [15.627536 , 15.627108 , 15.62758  , ..., 15.627242 ,
          15.627236 , 15.62723  ],
         [15.68143  , 15.681002 , 15.681474 , ..., 15.681136 ,
          15.68113  , 15.681124 ],
         ...,
         [16.54239  , 16.541962 , 16.542435 , ..., 16.542097 ,
          16.54209  , 16.542084 ],
         [16.62572  , 16.62529  , 16.625763 , ..., 16.625425 ,
          16.625418 , 16.625412 ],
         [16.384722 , 16.384293 , 16.384766 , ..., 16.384428 ,
          16.38442  , 16.384415 ]],

        [[15.72759  , 15.726189 , 15.726829 , ..., 15.726372 ,
          15.726365 , 15.726358 ],
         [15.628395 , 15.626994 , 15.627634 , ..., 15.627177 ,
          15.627171 , 15.627164 ],
         [15.682289 , 15.680888 , 15.681528 , ..., 15.681071 ,
          15.681065 , 15.681058 ],
...
         [16.017162 , 16.017141 , 16.013308 , ..., 16.012814 ,
          16.012808 , 16.012802 ],
         [16.015526 , 16.015505 , 16.011671 , ..., 16.011177 ,
          16.011171 , 16.011166 ],
         [16.01389  , 16.013868 , 16.010035 , ..., 16.00954  ,
          16.009535 , 16.00953  ]],

        [[15.990481 , 15.989479 , 15.986883 , ..., 15.986002 ,
          15.985996 , 15.985991 ],
         [15.992192 , 15.99119  , 15.988594 , ..., 15.987713 ,
          15.987707 , 15.987701 ],
         [15.99391  , 15.992908 , 15.990312 , ..., 15.98943  ,
          15.989425 , 15.989419 ],
         ...,
         [16.017294 , 16.01629  , 16.013697 , ..., 16.012815 ,
          16.01281  , 16.012804 ],
         [16.015657 , 16.014654 , 16.01206  , ..., 16.011179 ,
          16.011173 , 16.011168 ],
         [16.01402  , 16.013018 , 16.010424 , ..., 16.009542 ,
          16.009537 , 16.009531 ]]]], dtype=float32)
Coordinates:
    time     timedelta64[ns] 00:00:05.179000
    x        float64 4.5
    xu       float64 3.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 151.5 154.5 157.5 160.5 ... 289.5 292.5 295.5 298.5
  * yv       (yv) float64 648.0 651.0 654.0 ... 1.287e+03 1.29e+03 1.293e+03
    zw_3d_   (zw_3d) float64 153.0 156.0 159.0 162.0 ... 291.0 294.0 297.0 300.0
Dimensions without coordinates: zu_3d, zw_3d

In [1]:
temp

NameError: name 'temp' is not defined

In [ ]:
u2_2 = np.square(sub_ds.u)

In [58]:
%time u2.compute()

CPU times: user 10.3 s, sys: 27 s, total: 37.3 s
Wall time: 6.57 s


<xarray.DataArray 'u' (time: 72, zu_3d: 50, y: 216, xu: 192)>
array([[[[1.20187120e-03, 5.82910317e-04, 5.15852764e-04, ...,
          7.52276286e-09, 1.77709473e-04, 2.25754575e-05],
         [2.26918129e-08, 5.15691005e-04, 2.44591036e-03, ...,
          3.99560231e-04, 2.30569131e-04, 9.48354170e-07],
         [1.81410671e-03, 2.56682839e-03, 5.87040698e-03, ...,
          8.06077034e-04, 3.45382032e-05, 3.92679649e-05],
         ...,
         [5.80045511e-04, 2.13915337e-05, 9.17660000e-05, ...,
          1.92366529e-03, 1.84880849e-03, 4.38583374e-04],
         [7.68424070e-04, 2.60249805e-03, 2.35586049e-04, ...,
          1.08057319e-03, 2.12310508e-04, 9.75933654e-05],
         [2.22572125e-03, 2.58437311e-03, 5.14582149e-04, ...,
          5.32266240e-05, 8.35219225e-07, 4.84862685e-05]],

        [[1.13997507e-04, 3.34387482e-03, 3.35798359e-05, ...,
          9.64320824e-03, 3.21440655e-03, 1.30557048e-04],
         [4.80260613e-04, 3.38399317e-03, 1.92172464e-03, ...,
          6.77351188e-03, 3.63878487e-03, 2.10809149e-03],
         [2.98883038e-04, 4.52923588e-03, 8.46781023e-03, ...,
          2.79667112e-03, 1.22261979e-03, 1.10638421e-03],
...
          1.33965969e-07, 1.17135912e-07, 1.01032178e-07],
         [8.39452241e-08, 6.95892126e-08, 5.63059821e-08, ...,
          1.32427274e-07, 1.15471749e-07, 9.92742244e-08],
         [8.21748500e-08, 6.78160603e-08, 5.45703749e-08, ...,
          1.30878149e-07, 1.13817222e-07, 9.75445218e-08]],

        [[3.85853241e-07, 3.69492341e-07, 3.52311588e-07, ...,
          4.29273825e-07, 4.15809467e-07, 4.01315447e-07],
         [3.78335955e-07, 3.62651690e-07, 3.46150870e-07, ...,
          4.19762387e-07, 4.06955593e-07, 3.93126584e-07],
         [3.70271863e-07, 3.55286772e-07, 3.39489105e-07, ...,
          4.09642809e-07, 3.97512736e-07, 3.84369343e-07],
         ...,
         [7.91709027e-08, 6.60255921e-08, 5.38107834e-08, ...,
          1.23255532e-07, 1.07883110e-07, 9.31550659e-08],
         [7.75497639e-08, 6.44005453e-08, 5.22169756e-08, ...,
          1.21831846e-07, 1.06365896e-07, 9.15708469e-08],
         [7.59698153e-08, 6.28294572e-08, 5.06878379e-08, ...,
          1.20400955e-07, 1.04859083e-07, 9.00132093e-08]]]],
      dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 151.5 154.5 157.5 160.5 ... 289.5 292.5 295.5 298.5
Dimensions without coordinates: zu_3d

In [60]:
%timeit u2_2.compute()

6.73 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
u = sub_ds.u#.values
v = sub_ds.v#.values
w = sub_ds.w#.values
ubar_scalar = ne.evaluate("sqrt(u**2 + v**2 + w**2)")
print(time.time() - s)

20.63394570350647


In [39]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
u = sub_ds.u#.values
v = sub_ds.v#.values
w = sub_ds.w#.values
ubar_scalar = ne.evaluate("sqrt(u**2 + v**2 + w**2)")
print(time.time() - s)

22.61529040336609


In [5]:
ubar_scalar = ubar_scalar.mean(skipna=True)

<xarray.DataArray 'u' (time: 72, zu_3d: 193, y: 432, xu: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 1.5 4.5 7.5 10.5 13.5 ... 568.5 571.5 574.5 577.5
Dimensions without coordinates: zu_3d
Attributes:
    units:      m/s
    long_name:  u

In [ ]:
scalar_gradient = ds.s.mean(skipna=True)

In [11]:
TempSpacMeanU, TempSpacMeanV, TempSpacMeanW = calc_temp_spac_means(ds)

In [ ]:
uw = (ds.w - TempSpacMeanW) * (ds.u - TempSpacMeanU);
vw = (ds.w - TempSpacMeanW) * (ds.v - TempSpacMeanV);

In [ ]:
np.power(np.nanmean(uw)**2 + np.nanmean(vw)**2, 1/4)

In [13]:
ustar = (squeeze(nanmean(nanmean(uw,1),2).^2 + nanmean(nanmean(vw,1),2).^2)).^0.25;

KeyError: 'values'

In [8]:
ds.w

<xarray.DataArray 'w' (time: 72, zw_3d: 193, y: 432, x: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zw_3d_   (zw_3d) float64 3.0 6.0 9.0 12.0 15.0 ... 570.0 573.0 576.0 579.0
Dimensions without coordinates: zw_3d
Attributes:
    units:      m/s
    long_name:  w

In [ ]:
ubar = ubar_total

In [23]:
type(ubar_total)

numpy.ndarray

In [7]:
ne.evaluate("exp(2)")

array(7.3890561)

In [21]:
scalar_grad = ds.s.isel(y=y_domain, zu_3d=z_scalar)

In [16]:
ds.s.y.size * urban_ratio

216.0

In [6]:
ds.s.isel(time=60, zu_3d=10).v

array([[0.1304376 , 0.1125954 , 0.09213795, ..., 0.0807151 , 0.14683247,
        0.14305805],
       [0.12404612, 0.1070879 , 0.07839098, ..., 0.08024671, 0.13751815,
        0.1399681 ],
       [0.12678802, 0.12038978, 0.10291763, ..., 0.08745276, 0.13599472,
        0.13938259],
       ...,
       [0.12819828, 0.14347515, 0.18452443, ..., 0.0778394 , 0.13670161,
        0.14400712],
       [0.13251767, 0.12401052, 0.14600988, ..., 0.07967169, 0.15714346,
        0.1510945 ],
       [0.13553588, 0.11933067, 0.11963697, ..., 0.082677  , 0.15785772,
        0.14794137]], dtype=float32)

In [38]:
da = xr.DataArray(np.arange(25).reshape(5, 5), dims=("x", "y"))
da

<xarray.DataArray (x: 5, y: 5)>
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])
Dimensions without coordinates: x, y

In [37]:
tgt_x = xr.DataArray(np.arange(0, 5), dims="points")
tgt_y = xr.DataArray(np.arange(0, 5), dims="points")
da = da.isel(x=tgt_x, y=tgt_y)
da

ValueError: Dimensions {'y', 'x'} do not exist. Expected one or more of Frozen({'points': 5})

In [39]:
da.values

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [21]:
ds_lad.isel(zlad=1, y=431).lad.values

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.12297742, 0.12309078, 0.12335316, 0.12348082,
       0.1234502 , 0.1231009 , 0.12312585, 0.12326715, 0.12344291,
       0.12368529, 0.12351976, 0.12331436, 0.1232752 , 0.12315591,
       0.12324683, 0.12314666, 0.12312641, 0.12329989, 0.12336418,
       0.12303951, 0.12329254, 0.123154  , 0.12290037, 0.12301199,
       0.12321051, 0.12338139, 0.12368505, 0.12364503, 0.1232623 ,
       0.12320213, 0.12317088, 0.12309862, 0.1232764 , 0.1233467 ,
       0.12323636, 0.12299433, 0.12336692, 0.12379812, 0.12384361,
       0.12354115, 0.12339978, 0.12338317, 0.12345921, 0.12379106,
       0.12387869, 0.12366534, 0.12362086, 0.12342731, 0.12328358,
       0.12344657, 0.12358709, 0.12328151, 0.12307764, 0.12350874,
       0.12389586, 0.12396952, 0.12377783, 0.12331942, 0.12310649,
       0.1233413 , 0.12312192, 0.12304613, 0.12319905, 0.12308591,
       0.12295204, 0.12339766, 0.12366158, 0.12351003, 0.12327